In [1]:
#!/usr/bin/python3

import json
import requests
import pandas as pd
import ssl
import urllib.request
import geopandas as gpd 
import psycopg2

In [4]:
university = ["ru","bursa"]

for i in range(0,1):
    # Create an SSL context that doesn't verify the certificates.
    ssl_context = ssl.create_default_context()
    ssl_context.check_hostname = False
    ssl_context.verify_mode = ssl.CERT_NONE

    # Download the Field Mask from GeoNode
    filename = f"field_boundaries_{university[i]}"
    URL1 = "http://geoportal.addferti.eu/geoserver/ows"
    URL2 = "?service=WFS&version=1.0.0&request=GetFeature&typename=geonode%3A"
    URL4 = filename
    URL5 = "&outputFormat=json"
    URL = URL1 + URL2 + URL4 + URL5
    print(filename)
    print(URL)
    # Download the file.
    with urllib.request.urlopen(URL, context=ssl_context) as response, open(f"{filename}.geojson", "wb") as out_file:
        data = response.read()
        out_file.write(data)

    # Read the data using geopandas (simple feature = dataframe with geometry)
    sf_Field_Mask = gpd.read_file(f"{filename}.geojson")
    print(f"EPSG of {filename}: {sf_Field_Mask.crs.to_epsg()}")

field_boundaries_ru
http://geoportal.addferti.eu/geoserver/ows?service=WFS&version=1.0.0&request=GetFeature&typename=geonode%3Afield_boundaries_ru&outputFormat=json
EPSG of field_boundaries_ru: 32633


In [5]:
for i in range(0,1):
    postgreSQLTable = university[i] + "_soil_moisture"
    dsn_database = "postgres"    # Postgres databasename
    dsn_hostname = "127.0.0.1"
    dsn_port = "5432"
    dsn_uid = "postgres"    # Postgres username 
    dsn_pwd = "postgres"    # Postgres password 

    try:
        connec = psycopg2.connect(database=dsn_database,
                                   user=dsn_uid,
                                   password=dsn_pwd,
                                   host=dsn_hostname,
                                   port=dsn_port)
        print("Reading from Database", postgreSQLTable)
    except:
        print("Unable to connect to Database", university[i])

    # Select the most current soil moisture data
    SQL1 = "SELECT DISTINCT ON (device_id) device_id, soil_temp, soil_mc, soil_ec, lat, long FROM"
    SQL2 = postgreSQLTable
    SQL3 = "ORDER BY device_id, time desc;"
    SQL = SQL1 + " " + SQL2 + " " + SQL3

    cur = connec.cursor()
    cur.execute(SQL)
    sensor_df = cur.fetchall()
    print(sensor_df)
    
    


Reading from Database ru_soil_moisture
[('ru-lse-20', 20.11, 9.04, 61, 53.867291, 12.075781), ('ru-lse-25', 20.11, 10.09, 82, 53.86707517, 12.07613567), ('ru-lse-27', 15.67, 17.81, 310, 53.866935, 12.076494), ('ru-lse-28', 14.91, 31.77, 2592, 53.867105, 12.076745), ('ru-lse-29', 14.67, 27.69, 217, 53.866709, 12.077512), ('ru-lse-30', 15.31, 11.62, 0, 53.86776, 12.074829)]
